# Load csv data from drive 
Sources

- http://nali.org/load-google-drive-csv-panda-dataframe-google-colab/


In [ ]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
# Create local webserver and auto handles authentication.
gauth.LocalWebserverAuth()

In [2]:
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)

In [3]:
# scan through the files
folder_iterator = drive.ListFile({'q': "'1h-MNPZJg32SVFueMTFzpRG7b6QR2GXiE' in parents and trashed=false"})
folder_iterator

GoogleDriveFileList({'q': "'1h-MNPZJg32SVFueMTFzpRG7b6QR2GXiE' in parents and trashed=false"})

In [4]:
files = list(folder_iterator)
files[0][5]  # example file metadata

GoogleDriveFile({'embedLink': 'https://drive.google.com/file/d/1MT3XNh7o7KHgxwVQ5kziefv-FYHUxmcv/preview?usp=drivesdk', 'headRevisionId': '0B6HW1tAcpmsMTVRmOFJlWE5nN2ZwcVlTdENPTVp2V281OW93PQ', 'createdDate': '2018-09-20T13:51:30.780Z', 'fileSize': '3729', 'originalFilename': 'Rack_4_20_09_1551.csv', 'userPermission': {'id': 'me', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1MT3XNh7o7KHgxwVQ5kziefv-FYHUxmcv/permissions/me', 'type': 'user', 'kind': 'drive#permission', 'etag': '"qMp8o8Pju7Cka2UXv9ZwrYf1dpM/Kghnv2_BbW2C5B5vy_FmDE9ijBk"', 'role': 'writer'}, 'markedViewedByMeDate': '1970-01-01T00:00:00.000Z', 'etag': '"qMp8o8Pju7Cka2UXv9ZwrYf1dpM/MTUzNzQ1NTA4MDAwMA"', 'mimeType': 'text/csv', 'alternateLink': 'https://drive.google.com/file/d/1MT3XNh7o7KHgxwVQ5kziefv-FYHUxmcv/view?usp=drivesdk', 'parents': [{'kind': 'drive#parentReference', 'id': '1h-MNPZJg32SVFueMTFzpRG7b6QR2GXiE', 'parentLink': 'https://www.googleapis.com/drive/v2/files/1h-MNPZJg32SVFueMTFzpRG7b6QR2GXiE', 'selfLin

In [11]:
# example reading file directly
import pandas as pd
pd.set_option('display.max_rows', 10)
files[0][5].FetchContent()
pd.read_csv(files[0][5].content)

,Date,Time,Air temp,Humidity,Water level,Water temp,EC,pH,CO2,DO
0,20/09/2018,15:52:23,20.8,50.7,49.0,21.6,1.9,6.1,863.0,2.7
1,20/09/2018,15:53:23,20.8,50.5,48.0,21.6,1.9,6.1,778.0,2.5
2,20/09/2018,15:54:22,20.8,50.7,49.0,21.6,1.9,6.1,825.0,2.7
3,20/09/2018,15:55:22,20.7,50.4,49.0,21.6,1.9,6.1,810.0,2.6
4,20/09/2018,15:56:22,20.7,50.6,49.0,21.6,1.9,6.1,795.0,2.5
...,...,...,...,...,...,...,...,...,...,...
55,20/09/2018,16:47:20,20.4,50.8,48.0,21.3,1.9,6.1,789.0,2.7
56,20/09/2018,16:48:20,20.4,50.7,48.0,21.2,1.8,6.1,786.0,2.7
57,20/09/2018,16:49:20,20.5,50.2,48.0,21.2,1.9,6.1,829.0,2.7
58,20/09/2018,16:50:20,20.4,50.0,48.0,21.2,1.9,6.1,801.0,2.7


**Note:** Code below moved to ../shared-libraries/data_io.py

In [13]:
import re
import datetime
YEAR_DEFAULT = 2018

def parse_sensor_filename(fn):
    """
    Reads sensor file names and returns a dictionary of attributes contained
    in the filename.
    
    e.g. 'Rack_4_20_09_1551.csv'
    """
    matches = re.search(
        r'Rack_(?P<rack>\d)_(?P<day>\d+)_(?P<month>\d+)_(?P<hour>\d\d)(?P<minute>\d\d).csv', fn)
    if matches is None:
        if ".csv" in fn:
            print("csv error: ", fn)
        return None
    else:
        gd = matches.groupdict()
        dt = datetime.datetime(YEAR_DEFAULT, int(gd['month']),
               int(gd['day']), int(gd['hour']), int(gd['minute']))
        return {"FILE_TIMESTAMP": dt, "RACK": gd['rack']}
    
parse_sensor_filename('Rack_4_20_09_1551.csv')

{'FILE_TIMESTAMP': datetime.datetime(2018, 9, 20, 15, 51), 'RACK': '4'}

In [14]:
dfs = []
for files_chunk in files:
    for file in files_chunk:
        print(file['title'])
        filename_attributes = parse_sensor_filename(file['title'])
        if filename_attributes is not None:
            file.FetchContent()
            df = pd.read_csv(file.content)
            df['RACK'] = filename_attributes['RACK']
            df['FILE_TIMESTAMP'] = filename_attributes['FILE_TIMESTAMP']
            dfs.append(df)

byspireMonitoring_03_01-1355
Rack_4_20_09_1951.csv
Rack_4_20_09_1851.csv
Rack_4_20_09_1751.csv
Rack_4_20_09_1651.csv
Rack_4_20_09_1551.csv
Rack_4_20_09_1451.csv
Rack_4_20_09_1351.csv
Rack_4_20_09_1251.csv
Rack_4_20_09_1151.csv
Rack_4_20_09_1051.csv
Rack_4_20_09_0951.csv
Rack_4_20_09_0851.csv
Rack_4_20_09_0751.csv
Rack_4_20_09_0651.csv
Rack_4_20_09_0551.csv
Rack_4_20_09_0451.csv
Rack_4_20_09_0351.csv
Rack_4_20_09_0252.csv
Rack_4_20_09_0152.csv
Rack_4_20_09_0052.csv
Rack_4_19_09_2352.csv
Rack_4_19_09_2252.csv
Rack_4_19_09_2152.csv
Rack_4_19_09_2052.csv
Rack_4_19_09_1952.csv
Rack_4_19_09_1852.csv
Rack_4_19_09_1752.csv
Rack_4_19_09_1652.csv
Rack_4_19_09_1552.csv
Rack_4_19_09_1452.csv
Rack_2_19_09_1402.csv
Rack_4_19_09_1352.csv
Rack_2_19_09_1302.csv
Rack_4_19_09_1252.csv
Rack_2_19_09_1202.csv
Rack_4_19_09_1152.csv
Rack_2_19_09_1102.csv
Rack_4_19_09_1052.csv
Rack_2_19_09_1002.csv
Rack_4_19_09_0952.csv
Rack_2_19_09_0902.csv
Rack_4_19_09_0852.csv
Rack_2_19_09_0802.csv
Rack_4_19_09_0752.csv
Rac

ServerNotFoundError: Unable to find the server at doc-0o-0k-docs.googleusercontent.com

In [15]:
pd.concat(dfs)

,Date,Time,Air temp,Humidity,Water level,Water temp,EC,pH,CO2,DO,RACK,FILE_TIMESTAMP
0,20/09/2018,19:52:10,19.7,51.3,49.0,19.3,1.9,6.3,789.0,2.6,4,2018-09-20 19:51:00
1,20/09/2018,19:53:10,19.7,50.6,49.0,19.3,1.9,6.3,801.0,2.7,4,2018-09-20 19:51:00
2,20/09/2018,19:54:10,19.7,50.8,49.0,19.3,1.9,6.3,832.0,2.7,4,2018-09-20 19:51:00
3,20/09/2018,19:55:10,19.7,50.8,49.0,19.3,1.9,6.3,789.0,2.7,4,2018-09-20 19:51:00
4,20/09/2018,19:56:10,19.7,51.0,49.0,19.3,1.9,6.3,810.0,2.7,4,2018-09-20 19:51:00
...,...,...,...,...,...,...,...,...,...,...,...,...
55,16/09/2018,08:52:34,22.1,55.4,26.0,21.0,1.8,6.6,786.0,2.8,4,2018-09-16 07:56:00
56,16/09/2018,08:53:34,22.1,55.5,25.0,21.0,1.8,6.7,713.0,2.9,4,2018-09-16 07:56:00
57,16/09/2018,08:54:34,22.1,55.6,26.0,21.1,1.8,6.7,755.0,2.8,4,2018-09-16 07:56:00
58,16/09/2018,08:55:34,22.2,56.3,26.0,21.1,1.8,6.7,703.0,2.8,4,2018-09-16 07:56:00


### so far...
... google drive not the most elegant data lake solution, would probably better to have a cli that syncs up and down, and then analyze the data from disk. AWS S3 has some really nice functionality for this. Either way, we should look at coming up with something better than a single drive directory!